In [180]:
import pandas as pd
import json
import re
import numpy as np
import ast


In [181]:
# Load JSON file
with open('/home/rvissche/Nextcloud/What-If/data/json_structure_donations/json_structure_2_RB_Facebook.json', 'r') as f:
    data = json.load(f)

In [182]:
# Flatten JSON (handling nested structures)
df = pd.json_normalize(data, max_level=0)
df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

In [183]:
cols = df.columns[0:2775]
df = pd.melt(df, value_vars= cols)

In [184]:
df['path_1'] = df['variable'].str.split('/', n=1).str[0]
df['path_2'] = df['variable'].str.split('/', n=3).str[1]
df['path_3'] = df['variable'].str.split('/', n=3).str[2]
df['path_4'] = df['variable'].str.split('/', n=3).str[3]
df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


In [185]:
for index, row in df.iterrows():
    if isinstance(row['value'], list):
        df.at[index, 'value'] = row['value'][0]

In [186]:
df['json_1'] = ''
for index, row in df.iterrows():
    df.at[index, 'json_1'] = list(row['value'].keys())

df = df.explode('json_1')

In [187]:
df['json_2'] = df.apply(lambda row: [row['value'].get(row['json_1'], None)], axis=1)


In [188]:
df['json_2'] = df['json_2'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['json_2'] = df['json_2'].apply(lambda x: x[0] if isinstance(x, list) else x)
df = df.explode('json_2')


In [190]:
data_types = ['string', 'array', 'number', 'boolean']
print(type(data_types))

<class 'list'>


In [191]:
def process_json_2(row):
    if row['json_2'] in data_types:
        row['data_type'] = row['json_2']
        row['json_2'] = np.nan
    return row

df = df.apply(process_json_2, axis=1)


In [192]:
df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
       'path_3', 'path_4', 'json_name', 'json_1', 'json_2', 'data_type']]

In [178]:
for index, row in df.iterrows():
    if isinstance(row['json_2'], str) and row['json_2'] in data_types:
        df.at[index, 'data_type'] = row['json_2']
        df.at[index, 'json_2'] = np.nan
    